In [1]:
# Modelo YOLO Version 3 para detección de objetos

# Ruta al repositorio 
# C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/

# Ruta al fichero de configuracion yolov3.cfg
#C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/config

# Ruta a los pesos preentrenados yolov3.weights
# C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/weights/

# Importamos librerias

import torch
import torch.nn as nn
import sys
import os
import numpy as np 

print("Liberias importadas correctamente")

# Configuración de rutas
# Ruta donde hemos clonado el repositorio de Erik Lindernoren.
YOLOV3_REPO_PATH = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/'
YOLOV3_MODELS_PATH = os.path.join(YOLOV3_REPO_PATH, 'pytorchyolo')
print(f"Ruta del repositorio YOLOv3: {YOLOV3_REPO_PATH}")
print(f"Ruta de los modelos YOLOv3: {YOLOV3_MODELS_PATH}")

# Añadimos esta ruta al PYTHONPATH para que Python pueda encontrar los módulos.
sys.path.append(YOLOV3_REPO_PATH)
sys.path.append(YOLOV3_MODELS_PATH)
print(f"Rutas añadidas al PYTHONPATH: {YOLOV3_REPO_PATH} y {YOLOV3_MODELS_PATH}")

# Importamos las clases necesarias del repositorio.
# Darknet y YOLOLayer son las clases principales del modelo.
from models import Darknet, YOLOLayer 

# Parámetros Generales
# Número de clases del dataset BCCD (Glóbulos Rojos, Glóbulos Blancos, Plaquetas).
NUM_CLASSES_YOUR_DATASET = 3
# Tamaño de la imagen de entrada para el modelo YOLOv3 (típicamente 416x416 o 608x608).
IMG_SIZE = 416 

# Definimos los anchor masks para tus 3 clases (placeholder hasta definir los adecuados con KMeans)
# YOLOv3 usa 9 anchor boxes en total, divididos en 3 grupos de 3 para cada escala.
# Estos son los INDICES de los anchors. Los valores reales los calculaamos con K-Means.
# Ejemplo: si los 9 anchors se ordenan de menor a mayor área, los grandes (indices 6,7,8) van a la escala 13x13.
#Anchor Boxes Calculadas (Formato para YOLOv3Loss): [[(227, 210), (179, 155), (124, 111)], [(105, 113), (104, 96), (80, 109)], [(112, 75), (87, 82), (39, 38)]]

DUMMY_ANCHORS_MASKS = [
    [(227, 210), (179, 155), (124, 111)],  # Anchors para la escala más grande (stride 32, detecta objetos grandes)
    [(105, 113), (104, 96), (80, 109)],    # Anchors para la escala media (stride 16, detecta objetos medianos)
    [(112, 75), (87, 82), (39, 38)]        # Anchors para la escala más pequeña (stride 8, detecta objetos pequeños)
]

# Rutas de Archivos Específicos
# Archivo de configuracion yolov3.cfg
CONFIG_PATH = os.path.join(YOLOV3_REPO_PATH, 'config', 'yolov3.cfg')
CONFIG_PATH = CONFIG_PATH.replace('\\', '/')  # Asegúrate de usar barras normales para evitar problemas en Linux/Mac

# Archivo de pesos .weights descargado de https://github.com/patrick013/Object-Detection---Yolov3.git
WEIGHTS_PATH = os.path.join(YOLOV3_REPO_PATH, 'yolov3.weights')
WEIGHTS_PATH = WEIGHTS_PATH.replace('\\', '/')  # Asegúrate de usar barras normales para evitar problemas en Linux/Mac

# Detección del Dispositivo (CPU o GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Trabajando en el dispositivo: {device}")

# PASO 1: Instanciar el Modelo YOLOv3 (para 80 clases, usando el .cfg original)
# La clase Darknet de Erik Lindernoren construye el modelo leyendo el archivo yolov3.cfg.
# Esto crea el modelo con la arquitectura esperada por el archivo yolov3.weights.
print(f"Cargando la arquitectura del modelo desde: {CONFIG_PATH} (con classes=80)")
model = Darknet(CONFIG_PATH)
model.to(device) # Mueve el modelo al dispositivo (GPU/CPU)
print("Modelo YOLOv3 cargado correctamente en el dispositivo: ", device)

# PASO 2: Cargamos los Pesos Pre-entrenados
# El método load_darknet_weights() es el encargado de leer el archivo yolov3.weights.
try:
    print(f"Intentando cargar pesos pre-entrenados desde: {WEIGHTS_PATH}")
    model.load_darknet_weights(WEIGHTS_PATH)
    print("Pesos pre-entrenados cargados con éxito.")

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo de pesos en {WEIGHTS_PATH}.")
    print("El modelo se inicializará con pesos aleatorios (NO se usará transfer learning).")
    print("¡ADVERTENCIA! Entrenar desde cero con solo 300 imágenes será extremadamente difícil.")
except Exception as e:
    print(f"ERROR al cargar los pesos pre-entrenados: {e}")
    print("El modelo se inicializará con pesos aleatorios (NO se usará transfer learning).")
    print("¡ADVERTENCIA! Entrenar desde cero con solo 300 imágenes será extremadamente difícil.")

# ... (all your setup code above remains unchanged)

# PASO 3: Adapatacion del modelo para las 3 clases (FINE-TUNING EN MEMORIA)
print("\nAdaptando las capas de predicción a 3 clases...")

yolo_layer_index_in_model_yolo_layers = 0
for i, module_def in enumerate(model.module_defs):
    if module_def["type"] == "yolo":
        pred_conv_sequential_idx = i - 1
        pred_conv_layer_old = model.module_list[pred_conv_sequential_idx][0]
        yolo_layer_old_instance = model.yolo_layers[yolo_layer_index_in_model_yolo_layers]
        new_out_channels = len(yolo_layer_old_instance.anchors) * (5 + NUM_CLASSES_YOUR_DATASET)
        new_pred_conv_layer = nn.Conv2d(pred_conv_layer_old.in_channels, new_out_channels,
                                        kernel_size=pred_conv_layer_old.kernel_size,
                                        stride=pred_conv_layer_old.stride,
                                        padding=pred_conv_layer_old.padding,
                                        bias=True
                                        )
        model.module_list[pred_conv_sequential_idx] = nn.Sequential(new_pred_conv_layer)
        anchors_for_new_layer = yolo_layer_old_instance.anchors.tolist()
        stride_for_new_layer = yolo_layer_old_instance.stride
        new_yolo_layer = YOLOLayer(anchors_for_new_layer, NUM_CLASSES_YOUR_DATASET, new_coords=False)
        model.module_list[i] = nn.Sequential(new_yolo_layer)
        model.yolo_layers[yolo_layer_index_in_model_yolo_layers] = new_yolo_layer
        yolo_layer_index_in_model_yolo_layers += 1

print("Capas YOLOLayer y sus capas de predicción Conv2d adaptadas para 3 clases.")

# --- NUEVA SECCIÓN: CONGELAR TODAS LAS CAPAS, SOLO DESCONGELAR LAS ULTIMAS Conv2d DE PREDICCIÓN ---
print("\nConfigurando capas para Fine-Tuning (solo las capas Conv2d justo antes de cada YOLOLayer serán entrenables):")

# Congelar todos los parámetros por defecto
for param in model.parameters():
    param.requires_grad = False

# Solo las Conv2d antes de YOLOLayer quedan como entrenables
for i, module_def in enumerate(model.module_defs):
    if module_def["type"] == "yolo":
        pred_conv_sequential_idx = i - 1
        conv_seq = model.module_list[pred_conv_sequential_idx]
        # Buscar la capa Conv2d dentro del Sequential
        for layer in conv_seq:
            if isinstance(layer, nn.Conv2d):
                for param in layer.parameters():
                    param.requires_grad = True
                print(f"  Descongelada capa Conv2d antes del YOLOLayer en module_list[{i}]")

# --- Verificación de Capas Entrenables ---
print("\nVerificación de capas que se entrenarán ('requires_grad=True'):")
trainable_params_count = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        trainable_params_count += param.numel()

total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal de parámetros entrenables: {trainable_params_count / 1e6:.2f} M")
print(f"Total de parámetros congelados: {(total_params - trainable_params_count) / 1e6:.2f} M")
print(f"Total de parámetros en el modelo: {total_params / 1e6:.2f} M")

# PASO 5: Prueba Final de la Pasada hacia Adelante (sanity check)
print("\nRealizando una pasada hacia adelante para verificar la configuración del modelo...")

model.eval()
dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)
with torch.no_grad():
    predictions = model(dummy_input)

print(f"\nShape de la salida del modelo después de cargar pesos y adaptar a {NUM_CLASSES_YOUR_DATASET} clases (en modo EVAL):")
print(f"  Escala 13x13: {predictions[0].shape} (Esperado: [N, 3*13*13, 5+C])")
print(f"  Escala 26x26: {predictions[1].shape} (Esperado: [N, 3*26*26, 5+C])")
print(f"  Escala 52x52: {predictions[2].shape} (Esperado: [N, 3*52*52, 5+C])")
print(f"¡Las dimensiones de salida para {NUM_CLASSES_YOUR_DATASET} clases son correctas en modo EVAL!\n")
print("\n--- ¡Fase de Configuración del Modelo YOLOv3 Completada Exitosamente! ---")


Liberias importadas correctamente
Ruta del repositorio YOLOv3: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/
Ruta de los modelos YOLOv3: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/pytorchyolo
Rutas añadidas al PYTHONPATH: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/ y C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/pytorchyolo
Trabajando en el dispositivo: cpu
Cargando la arquitectura del modelo desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/config/yolov3.cfg (con classes=80)
Modelo YOLOv3 cargado correctamente en el dispositivo:  cpu
Intentando cargar pesos pre-entrenados desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/yolov3.weights
Pesos pre-entrenados cargados con éxito.

Adaptando las capas de predicción a 3 clases...
Capas YOLOLayer y sus capas de predicción Conv2d adaptadas para 3 clases.

Configurando capas para Fine-Tuning (solo las capas Conv2d justo antes de cada YOLOLayer serán entr